# Aula de algoritmos de segmentação de dados

Nesta aula vamos colocar às mãos no código para ter uma visão prática do uso de algoritmos de aprendizado de máquina para extrair um entendimento mais profundo de uma base de dados.

Neste exemplo, vamos usar a base de dados que sumariza 82 características de 21.263 materiais supercondutores diferentes. Esses dados foram compilados de artigos científicos pelo Superconducting Material Database maintained by Japan’s National Institute for Materials Science (NIMS), e são disponíveis Materials Data Repository (MDR).

A tabela que utilizaremos é uma versão compilada e disponível em: https://www.kaggle.com/datasets/munumbutt/superconductor-dataset?select=train.csv

## Importando bibliotecas e dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

## Preparando o ambiente

In [ ]:
## Preparando o ambiente
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/tiagofiorini/MLinPhysics.git
    import os as os
    os.chdir('./MLinPhysics')

In [ ]:
dados = pd.read_csv('Superconductivity.csv')
dados

O nome atribuído às 82 características dos materiais são:

In [ ]:
dados.columns

**Tarefa 1:** Explore a estatística dos dados por meio de histogramas.

*Nota:* Copie o nome da coluna e substitua no código abaixo!

In [ ]:
plt.hist(dados['mean_ThermalConductivity'], bins=20);

## Verificando a existência de agrupamentos!

Vamos verificar possíveis agrupamentos entre duas variáveis: a afinidade eletrônica e a temperatura crítica de supercondutividade.

Faremos isso por meio de um histograma bidimensional!

In [ ]:
plt.hist2d(dados['mean_ElectronAffinity'], dados['critical_temp'],bins=50);
plt.xlabel('Afinidade eletrônica')
plt.ylabel('Temperatura crítica');

## Segmentação dos dados

Vamos utilizar algoritmos de clusterização para segmentar esses dados.

Carregando a biblioteca correspondente:

In [ ]:
from sklearn.cluster import KMeans 

Verifique a seguir a variável *n_clusters* que define o número de agrupamentos a se calcular.

**Tarefa:** Variar o número de agrupamentos e verificar a influência na resposta do algoritmo.

*Nota:* Para manter a funcionalidadde das células a seguir, prossiga apenas de *n_clusters* for maior que 2 e menor que 4! 

In [ ]:
n_clusters= 2

X = np.array([dados['mean_ElectronAffinity'], dados['critical_temp']]).T
kmeans = KMeans(n_clusters=n_clusters).fit(X)
categorias = kmeans.labels_

plt.scatter(dados['mean_ElectronAffinity'], dados['critical_temp'], c=categorias)
plt.xlabel('Afinidade eletrônica')
plt.ylabel('Temperatura crítica');

*Nota:* Verifique como o Kmeans nomeia as categorias:

In [ ]:
np.unique(categorias)

Para uma melhor visualização, vamos utilizar um histograma bidimensional.

In [ ]:
import matplotlib.colors as mcolors

colors = [(1,0,0,c) for c in np.linspace(0,1,100)]
cmapred  = mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0,1,0,c) for c in np.linspace(0,1,100)]
cmapgreen  = mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0,0,1,c) for c in np.linspace(0,1,100)]
cmapblue =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(1,0,1,c) for c in np.linspace(0,1,100)]
cmappurple =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0,1,1,c) for c in np.linspace(0,1,100)]
cmapcian =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)

colormaps = [cmapred, cmapgreen, cmapblue, cmappurple, cmapcian]
for i in range(categorias.max()+1):
    mask = ( categorias == i )
    x = X[mask,:]
    plt.hist2d(x[:,0],x[:,1],bins=[np.linspace(0,300,50),np.linspace(0,200,50)],cmap=colormaps[i])
plt.xlabel('Afinidade eletrônica')
plt.ylabel('Temperatura crítica');

## Vamos testar a proposta de segmentação?

Para seguir, execute novamente a clusterização em um número de agrupamentos a se testar!

Carregando o teste de silhueta:

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

silhouette_avg = silhouette_score(X, categorias)

Veja o fator de silhueta médio para todos os pontos:

*Nota 1:* Um fator de silhueta mais próximo de 1 indica uma boa segmentação do ponto de vista estatístico!

*Nota 2:* conforme observado nos histogramas, não podemos esperar uma segmentação perfeita!

In [ ]:
silhouette_avg

Vamos analisar ponto a ponto a análise da silhueta!

In [ ]:
import matplotlib.cm as cm

# Compute the silhouette scores for each sample
sample_silhouette_values = silhouette_samples(X, categorias)
y_lower = 10

for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[categorias == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        plt.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
    
plt.axvline(x=silhouette_avg, color="red", linestyle="--");

*Nota :* Qual a porcentagem de dados mal classificados nos diferentes agrupamentos?

In [ ]:
mask = ( sample_silhouette_values < 0 )
p = (100 * mask.sum() / len(mask)).round(2)
print('Apenas %.2f%% dos pontos foram mal classificados nos agrupamentos de dados.'%(p))

##Podemos interpretar os resultados?

Podemos usar os centros dos agrupamentos para tentar extrair alguma conclusão sobre a análise (*insight*).

In [ ]:
centros = kmeans.cluster_centers_
dados_provisorio = pd.DataFrame(data=centros, columns=['mean_ElectronAffinity','critical_temp'])
dados_provisorio

In [ ]:
ordem = centros[:,0].argsort()
plt.plot(centros[ordem,0],centros[ordem,1])
plt.xlabel('Afinidade eletrônica')
plt.ylabel('Temperatura crítica');